In [1]:
import model
import torch
import dataset
import model
import math
import numpy as np
import utility
import os


In [12]:
def test_reg_model(model, test_dataset, batch_size=50, criterion=None, num_rects=4, screen_w=640, screen_h=480):
    model.eval()
    
    if criterion is None:
        criterion = torch.nn.MSELoss().cuda()

    print("CURRENT MODEL seq_len: {}".format(test_dataset.seq_len))
    print("CURRENT MODEL: {}".format(model.__class__.__name__))
    print("CURRENT DATASET SIZE: {}".format(test_dataset.__len__()))
    print("CURRENT NUM RECTS: {}".format(num_rects))
    
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    min_error = 999999
    max_error = 0
    sum_error = 0
    count_error = 0
    errors = np.zeros(test_dataset.__len__())

    correct = 0
    total = 0
    
    min_loss = 999999
    max_loss = 0
    sum_loss = 0
    count_loss = 0

    with torch.no_grad():
        for i, (eye_left, eye_right, face, pos) in enumerate(test_loader):
            eye_left = eye_left.to(device)
            eye_right = eye_right.to(device)
            face = face.to(device)
            pos = pos.to(device)

            # Forward pass
            out = model(eye_left, eye_right, face)
            pos = pos[:, -1, :]
            loss = criterion(out, pos)

            out = out.cpu().detach().numpy()
            pos = pos.cpu().numpy()

            for b in range(pos.shape[0]):
                o = out[b]
                p = pos[b]

                o_x = math.ceil((o[0]) / (screen_w / num_rects))
                o_y = math.ceil((o[1]) / (screen_h / num_rects))

                p_x = math.ceil((p[0]) / (screen_w / num_rects))
                p_y = math.ceil((p[1]) / (screen_h / num_rects))
                
                error = math.sqrt((o_x - p_x)**2 + (o_y - p_y)**2)

                if error < min_error:
                    min_error = error
                if error > max_error:
                    max_error = error
                sum_error += error

                errors[count_error] = error
                
            
                
                if ((p_y - 1) * num_rects + (p_x - 1)) == ((o_y - 1) * num_rects + (o_x - 1)):
                    correct += 1
                    
                total += 1
                
                count_error += 1

            if loss.item() < min_loss:
                min_loss = loss.item()
            if loss.item() > max_loss:
                max_loss = loss.item()
            sum_loss += loss.item()
            count_loss += 1

        # print('MIN MSELoss: {} '.format(min_loss))
        # print('MAX MSELoss: {} '.format(max_loss))
        # print('AVG MSELoss: {}'.format(sum_loss / count_loss))

        print('MIN error: {} '.format(min_error))
        print('MAX error: {} '.format(max_error))
        print('AVG error: {}'.format(sum_error / count_error))
        print('MEAD error: {}'.format(np.median(errors)))

    print('Accuracy : %d %%' % (100 * correct / total))
    return test_dataset.__len__(), min_error, max_error, (sum_error / count_error), (np.median(errors)), errors


In [17]:
def test_classifire_model(model, test_dataset, batch_size=50):
    model.eval()  
    
    criterion = torch.nn.BCELoss().cuda()

    print("CURRENT MODEL seq_len: {}".format(test_dataset.seq_len))
    print("CURRENT MODEL: {}".format(model.__class__.__name__))
    print("CURRENT DATASET SIZE: {}".format(test_dataset.__len__()))
    print("CURRENT NUM RECTS: {}".format(num_rects))

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    correct = 0
    total = 0

    min_loss = 999999
    max_loss = 0
    sum_loss = 0
    count_loss = 0

    with torch.no_grad():
        for i, (eye_left, eye_right, face, pos) in enumerate(test_loader):
            eye_left = eye_left.to(device)
            eye_right = eye_right.to(device)
            face = face.to(device)
            pos = pos.to(device)

            # Forward pass
            out = model(eye_left, eye_right, face)
            loss = criterion(out, pos)

            _, predicted = torch.max(out.data, 1)
            _, pos = torch.max(pos.data, 1)
            print(pos)
            total += pos.size(0)
            
            correct += (predicted == pos).sum().item()
            if loss.item() < min_loss:
                min_loss = loss.item()
            if loss.item() > max_loss:
                max_loss = loss.item()
            sum_loss += loss.item()
            count_loss += 1

        
        # print('MIN BCELoss: {} '.format(min_loss))
        # print('MAX BCELoss: {} '.format(max_loss))
        # print('AVG BCELoss: {}'.format(sum_loss / count_loss))
        
    print('Accuracy : %d %%' % (100 * correct / total))

In [18]:
def test_class_reg_model(model, classifier_model, test_dataset, batch_size=50, criterion=None, num_rects=4, screen_w=640, screen_h=480):
    model.eval()
    classifier_model.eval()

    if criterion is None:
        criterion = torch.nn.MSELoss().cuda()

    print("CURRENT MODEL seq_len: {}".format(test_dataset.seq_len))
    print("CURRENT MODEL: {}".format(model.__class__.__name__))
    print("CURRENT DATASET SIZE: {}".format(test_dataset.__len__()))
    print("CURRENT NUM RECTS: {}".format(num_rects))

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    min_error = 999999
    max_error = 0
    sum_error = 0
    count_error = 0
    errors = np.zeros(test_dataset.__len__())

    correct = 0
    total = 0

    min_loss = 999999
    max_loss = 0
    sum_loss = 0
    count_loss = 0

    with torch.no_grad():
        for i, (eye_left, eye_right, face, pos) in enumerate(test_loader):
            eye_left = eye_left.to(device)
            eye_right = eye_right.to(device)
            face = face.to(device)
            pos = pos.to(device)

            # Forward pass
            classes = classifier_model(eye_left, eye_right, face)
            out = model(eye_left, eye_right, face, classes)
            pos = pos[:, -1, :]
            loss = criterion(out, pos)

            out = out.cpu().detach().numpy()
            pos = pos.cpu().numpy()

            for b in range(pos.shape[0]):
                o = out[b]
                p = pos[b]

                o_x = math.ceil((o[0]) / (screen_w / num_rects))
                o_y = math.ceil((o[1]) / (screen_h / num_rects))

                p_x = math.ceil((p[0]) / (screen_w / num_rects))
                p_y = math.ceil((p[1]) / (screen_h / num_rects))
                
                error = math.sqrt((o_x - p_x)**2 + (o_y - p_y)**2)

                if error < min_error:
                    min_error = error
                if error > max_error:
                    max_error = error
                sum_error += error

                errors[count_error] = error
                
            
                
                if ((p_y - 1) * num_rects + (p_x - 1)) == ((o_y - 1) * num_rects + (o_x - 1)):
                    correct += 1
                    
                total += 1
                
                count_error += 1

            if loss.item() < min_loss:
                min_loss = loss.item()
            if loss.item() > max_loss:
                max_loss = loss.item()
            sum_loss += loss.item()
            count_loss += 1

        # print('MIN MSELoss: {} '.format(min_loss))
        # print('MAX MSELoss: {} '.format(max_loss))
        # print('AVG MSELoss: {}'.format(sum_loss / count_loss))

        print('MIN error: {} '.format(min_error))
        print('MAX error: {} '.format(max_error))
        print('AVG error: {}'.format(sum_error / count_error))
        print('MEDIAN error: {}'.format(np.median(errors)))

    print('Accuracy : %d %%' % (100 * correct / total))
    return test_dataset.__len__(), min_error, max_error, (sum_error / count_error), (np.median(errors)), errors


In [34]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

dataset_seq_len = 60

batch_size = 10

num_rects = 16
seq_len = 8

max_samples = 10770 


In [35]:
m = utility.load_model('./models/TwoEyes/model_{}_1.pth'.format(
    max_samples), 
            device,
            model.TwoEyes(seq_len=1))

d = dataset.Dataset(seq_len=1, dirname='./test')

test_reg_model(m, d, batch_size=batch_size, num_rects=num_rects)

./test/60/
1178
LOADING DATA...


LOAD MODEL: 
./models/TwoEyes/model_10770_1.pth
FOUND MODEL
0


END LOAD DATA


1000
CURRENT MODEL seq_len: 1
CURRENT MODEL: TwoEyes
CURRENT DATASET SIZE: 1178
CURRENT NUM RECTS: 16
MIN error: 0.0 
MAX error: 188996.0805757622 
AVG error: 703.5967326836037
MEAD error: 2.23606797749979
Accuracy : 5 %


(1178,
 0.0,
 188996.0805757622,
 703.5967326836037,
 2.23606797749979,
 array([1.        , 1.41421356, 1.        , ..., 1.41421356, 3.60555128,
        0.        ]))

In [36]:
m = utility.load_model('./models/TwoEyesLSTM/model_{}_{}.pth'.format(
    max_samples, seq_len), 
            device,
            model.TwoEyesLSTM(seq_len=seq_len))

d = dataset.Dataset(seq_len=seq_len, dirname='./test')

test_reg_model(m, d, batch_size=batch_size, num_rects=num_rects)

LOAD MODEL: 
./models/TwoEyesLSTM/model_10770_8.pth
FOUND MODEL


./test/60/
1178
LOADING DATA...


0


END LOAD DATA


1000
CURRENT MODEL seq_len: 8
CURRENT MODEL: TwoEyesLSTM
CURRENT DATASET SIZE: 1178
CURRENT NUM RECTS: 16
MIN error: 0.0 
MAX error: 19.1049731745428 
AVG error: 2.1026770784247817
MEAD error: 1.4142135623730951
Accuracy : 8 %


(1178,
 0.0,
 19.1049731745428,
 2.1026770784247817,
 1.4142135623730951,
 array([1.41421356, 2.82842712, 3.        , ..., 2.        , 3.16227766,
        1.41421356]))

In [ ]:
m = utility.load_model('./models/EyeClassifierLSTM/model_{}_{}_{}.pth'.format(
    max_samples, num_rects * num_rects, seq_len), 
                           device,
                           model.EyeClassifierLSTM(num_rects * num_rects, seq_len))

d = dataset.CDataset(num_rects=num_rects, seq_len=seq_len, dirname='./test')

test_classifire_model(m, d, batch_size)

In [37]:
c_m = utility.load_model('./models/EyeClassifierLSTM/model_{}_{}_{}.pth'.format(
        max_samples, num_rects * num_rects, seq_len),
        device,
        model.EyeClassifierLSTM(num_rects * num_rects, seq_len))

m = utility.load_model('./models/EyesLSTMWithClassifier/model_{}_{}_{}.pth'.format(
    max_samples, seq_len, num_rects), 
            device,
            model.EyesLSTMWithClassifier(seq_len=seq_len, num_rects=(num_rects * num_rects)))

d = dataset.Dataset(seq_len=seq_len, dirname='./test')

test_class_reg_model(m, c_m, d, batch_size=batch_size, num_rects=num_rects)

LOAD MODEL: 
./models/EyeClassifierLSTM/model_10770_256_8.pth
FOUND MODEL
LOAD MODEL: 
./models/EyesLSTMWithClassifier/model_10770_8_16.pth
FOUND MODEL


./test/60/
1178
LOADING DATA...


0


END LOAD DATA


1000
CURRENT MODEL seq_len: 8
CURRENT MODEL: EyesLSTMWithClassifier
CURRENT DATASET SIZE: 1178
CURRENT NUM RECTS: 16
MIN error: 0.0 
MAX error: 190.5465822312224 
AVG error: 2.227015535640416
MEDIAN error: 1.4142135623730951
Accuracy : 13 %


(1178,
 0.0,
 190.5465822312224,
 2.227015535640416,
 1.4142135623730951,
 array([0.        , 1.        , 7.61577311, ..., 0.        , 2.23606798,
        2.82842712]))

In [ ]:
import cv2
import scipy.misc
%matplotlib inline
import matplotlib.pyplot as plt

d = dataset.Dataset(seq_len=8)
print(d.eye_left[0][4].shape)
img = d.eye_left[0][4].transpose(1, 2, 0)
'''
print(img.shape)
print(d.eye_left[0][4] == img)
i = np.zeros((32, 32 ,3))
plt.imshow(img)
for i in range (0, 4):
    img = d.eye_left[0][i].transpose(1, 2, 0)
    plt.figure()
    plt.imshow(img)
    
'''  
img = d.eye_left[0][0].transpose(1, 2, 0)
scipy.misc.imsave('left_exapmle.png'.format(i), img)

img = d.eye_right[0][0].transpose(1, 2, 0)
scipy.misc.imsave('right_exapmle.png'.format(i), img)


img = d.eye_left[0][0].transpose(1, 2, 0)
img = np.concatenate((img, d.eye_right[0][1].transpose(1, 2, 0)), axis=1)
scipy.misc.imsave('two_exapmle.png'.format(i), img)


img = d.eye_left[0][0].transpose(1, 2, 0)
img = np.concatenate((img, d.eye_left[0][1].transpose(1, 2, 0)), axis=1)
img = np.concatenate((img, d.eye_left[0][2].transpose(1, 2, 0)), axis=1)
img = np.concatenate((img, d.eye_left[0][3].transpose(1, 2, 0)), axis=1)
scipy.misc.imsave('goritjopa.jpg'.format(i), img)
plt.imshow(img)